In [1]:
import os, sys
from pathlib import Path
PROJECTPATH = Path(".").cwd().parent.absolute()
sys.path.insert(0, PROJECTPATH)
os.chdir(PROJECTPATH)
PROJECTPATH

PosixPath('/home/jmgibso3/ncstate/projects/auth_checker')

In [2]:
# from casbin_redis_adapter.adapter import Adapter
from casbin_pymongo_adapter.adapter import Adapter
from auth_checker.util.settings import CASBIN_RBAC_MODEL
import casbin
from casbin.model import Model

model = Model()
model.load_model_from_text(CASBIN_RBAC_MODEL)

adapter = Adapter(os.getenv('MONGODB_URI'), dbname=os.getenv('MONGODB_DB'))
e = casbin.Enforcer(model, adapter, True)

TypeError: name must be an instance of str

In [5]:
e.get_all_named_subjects("jmgibso3")

[]

In [6]:
def set_policy(sub, obj=None, act=None, group=None):
    if group:
        return e.add_role_for_user(sub, group)
    return e.add_policy(sub, obj, act)

In [7]:
def group_policies():
    set_policy('clearance_liaison', 'clearance', 'read')
    set_policy('clearance_admin', 'clearance', 'read')
    set_policy('clearance_admin', 'clearance', 'write')
    set_policy('clearance_dev', 'clearance', 'assume')

def user_policies():
    set_policy('jmgibso3', group='clearance_admin')
    set_policy('jmgibso3', group='clearance_dev')
    set_policy('jtchampi', group='clearance_liaison')
    set_policy('dgreen', group='clearance_admin')
    set_policy('clearance_dev', group='clearance_admin')

In [8]:
group_policies()

In [9]:
user_policies()

In [7]:
e.get_implicit_permissions_for_user('jmgibso3')

[['clearance_admin', 'clearance', 'read'],
 ['clearance_admin', 'clearance', 'write'],
 ['clearance_dev', 'clearance', 'assume']]

In [8]:
e.get_roles_for_user('jtchampi')

['clearance_liaison']

In [8]:
e.get_implicit_permissions_for_user('jtchampi')

[['clearance_liaison', 'clearance', 'read']]

In [40]:
e.enforce('jtchampi', 'clearance', 'assume')

2024-04-08 10:15:25,196 Request: jtchampi, clearance, assume ---> True


True

In [41]:
e.get_implicit_permissions_for_user('dgreen')

[['clearance_admin', 'clearance', 'read'],
 ['clearance_admin', 'clearance', 'write']]

In [44]:
e.enforce('dgreen', 'clearance', 'assume')

2024-04-08 10:17:39,688 Request: dgreen, clearance, assume ---> False


False

# RBAC Model
-------------
[request_definition]
r = sub, obj, act

[policy_definition]
p = sub, obj, act

[role_definition]
g = _, _

[policy_effect]
e = some(where (p.eft == allow))

[matchers]
m = g(r.sub, p.sub) && r.obj == p.obj && r.act == p.act

-----------

# Policy Examples
-----------------

p, alice, data1, read

p, bob, data2, write

p, data2_admin, data2, read

p, data2_admin, data2, write

g, alice, data2_admin

----------------

## In Words

When creating RBAC policies there are two possibilities `p` and `g`. 

A `p` will always have an action (e.g. 'write', 'read', 'update', 'delete', 'give_raspberries')
A `g` will always have a user or (maybe? another group) as the subject and a group as the object

# Groups
* owner
* liaison
  * liaison_authorizor
  * liaison_requestor
* cmt
* automations

## Policies
* liaison_admin
  * (p, liaison_admin, liaison, read)
  * (p, liaison_admin, liaison, write)


[]